In [1]:
# Importing necessary packages
import pandas as pd
import numpy as np
import json
import nltk
import re
from nltk.corpus import stopwords
from langdetect import detect
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
# Uploading Dataset and inspecting its contents
df = pd.DataFrame()
df = pd.read_csv('/Users/dwreiter/Desktop/Work/Springboard/Video Game Capstone Project/Data Wrangling/Video_Games_Sales_as_at_22_Dec_2016.csv', index_col=0)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
Name,,,,,,,,,,,,,,,
Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Dropped all missing values
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6825 entries, Wii Sports to STORM: Frontline Nation
Data columns (total 15 columns):
Platform           6825 non-null object
Year_of_Release    6825 non-null float64
Genre              6825 non-null object
Publisher          6825 non-null object
NA_Sales           6825 non-null float64
EU_Sales           6825 non-null float64
JP_Sales           6825 non-null float64
Other_Sales        6825 non-null float64
Global_Sales       6825 non-null float64
Critic_Score       6825 non-null float64
Critic_Count       6825 non-null float64
User_Score         6825 non-null object
User_Count         6825 non-null float64
Developer          6825 non-null object
Rating             6825 non-null object
dtypes: float64(9), object(6)
memory usage: 853.1+ KB


In [4]:
# Changed Release Year to integers
df['Year_of_Release'] = (df['Year_of_Release']).astype(int)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
Name,,,,,,,,,,,,,,,
Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
New Super Mario Bros.,DS,2006,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E
Wii Play,Wii,2006,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E


In [5]:
# Seeing how many types of genres there are
df['Genre'].value_counts()

Action          1630
Sports           943
Shooter          864
Role-Playing     712
Racing           581
Platform         403
Misc             384
Fighting         378
Simulation       297
Strategy         267
Adventure        248
Puzzle           118
Name: Genre, dtype: int64

In [6]:
# Making columns for each type of game
df['Action'] = (df['Genre'] == 'Action').astype(int)
df['Adventure'] = (df['Genre'] == 'Adventure').astype(int)
df['Fighter'] = (df['Genre'] == 'Fighting').astype(int)
df['Platformer'] = (df['Genre'] == 'Platform').astype(int)
df['Puzzler'] = (df['Genre'] == 'Puzzle').astype(int)
df['Racing'] = (df['Genre'] == 'Racing').astype(int)
df['RPG'] = (df['Genre'] == 'Role-Playing').astype(int)
df['Shooter'] = (df['Genre'] == 'Shooter').astype(int)
df['Simulation'] = (df['Genre'] == 'Simulation').astype(int)
df['Sports'] = (df['Genre'] == 'Sports').astype(int)
df['Strategy'] = (df['Genre'] == 'Strategy').astype(int)
df['Variety'] = (df['Genre'] == 'Misc').astype(int)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,...,Fighter,Platformer,Puzzler,Racing,RPG,Shooter,Simulation,Sports,Strategy,Variety
Name,,,,,,,,,,,,,,,,,,,,,
Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,...,0,0,0,0,0,0,0,1,0,0
Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,...,0,0,0,1,0,0,0,0,0,0
Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,...,0,0,0,0,0,0,0,1,0,0
New Super Mario Bros.,DS,2006,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,...,0,1,0,0,0,0,0,0,0,0
Wii Play,Wii,2006,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
# Seeing how many types of consoles there are
df['Platform'].value_counts()

PS2     1140
X360     858
PS3      769
PC       651
XB       565
Wii      479
DS       464
PSP      390
GC       348
PS4      239
GBA      237
XOne     159
3DS      155
PS       150
PSV      118
WiiU      89
DC        14
Name: Platform, dtype: int64

In [8]:
# Making columns for some of the consoles
df['Game Boy Advance'] = (df['Platform'] == 'GBA').astype(int)
df['Nintendo 3DS'] = (df['Platform'] == '3DS').astype(int)
df['Nintendo DS'] = (df['Platform'] == 'DS').astype(int)
df['Nintendo GameCube'] = (df['Platform'] == 'GC').astype(int)
df['Nintendo Wii'] = (df['Platform'] == 'Wii').astype(int)
df['Nintendo Wii U'] = (df['Platform'] == 'WiiU').astype(int)
df['PC'] = (df['Platform'] == 'PC').astype(int)
df['PlayStation'] = (df['Platform'] == 'PS').astype(int)
df['PlayStation 2'] = (df['Platform'] == 'PS2').astype(int)
df['PlayStation 3'] = (df['Platform'] == 'PS3').astype(int)
df['PlayStation 4'] = (df['Platform'] == 'PS4').astype(int)
df['PlayStation Vita'] = (df['Platform'] == 'PSV').astype(int)
df['PSP'] = (df['Platform'] == 'PSP').astype(int)
df['Sega Dreamcast'] = (df['Platform'] == 'DC').astype(int)
df['Xbox'] = (df['Platform'] == 'XB').astype(int)
df['Xbox 360'] = (df['Platform'] == 'X360').astype(int)
df['Xbox One'] = (df['Platform'] == 'XOne').astype(int)
df.head()

,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,...,PlayStation,PlayStation 2,PlayStation 3,PlayStation 4,PlayStation Vita,PSP,Sega Dreamcast,Xbox,Xbox 360,Xbox One
Name,,,,,,,,,,,,,,,,,,,,,
Wii Sports,Wii,2006,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,...,0,0,0,0,0,0,0,0,0,0
Mario Kart Wii,Wii,2008,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,...,0,0,0,0,0,0,0,0,0,0
Wii Sports Resort,Wii,2009,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,...,0,0,0,0,0,0,0,0,0,0
New Super Mario Bros.,DS,2006,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,...,0,0,0,0,0,0,0,0,0,0
Wii Play,Wii,2006,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Saving DataFame to CSV
df.to_csv('vg_df.csv', sep='\t')

In [10]:
# Uploading second Dataset and inspecting its contents
df_2 = pd.read_csv('/Users/dwreiter/Desktop/Work/Springboard/Video Game Capstone Project/Data Wrangling/metacritic_game_user_comments.csv', index_col=0)
df_2.head()

,Title,Platform,Userscore,Comment,Username
0,The Legend of Zelda: Ocarina of Time,Nintendo64,10,"Everything in OoT is so near at perfection, it...",SirCaestus
1,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I won't bore you with what everyone is already...,Kaistlin
2,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Anyone who gives the masterpiece below a 7 or ...,Jacody
3,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I'm one of those people who think that this is...,doodlerman
4,The Legend of Zelda: Ocarina of Time,Nintendo64,10,This game is the highest rated game on Metacr...,StevenA


In [11]:
# Dropped all missing values
df_2 = df_2.dropna()
df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283957 entries, 0 to 283982
Data columns (total 5 columns):
Title        283957 non-null object
Platform     283957 non-null object
Userscore    283957 non-null int64
Comment      283957 non-null object
Username     283957 non-null object
dtypes: int64(1), object(4)
memory usage: 13.0+ MB


In [12]:
df_2.head()

,Title,Platform,Userscore,Comment,Username
0,The Legend of Zelda: Ocarina of Time,Nintendo64,10,"Everything in OoT is so near at perfection, it...",SirCaestus
1,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I won't bore you with what everyone is already...,Kaistlin
2,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Anyone who gives the masterpiece below a 7 or ...,Jacody
3,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I'm one of those people who think that this is...,doodlerman
4,The Legend of Zelda: Ocarina of Time,Nintendo64,10,This game is the highest rated game on Metacr...,StevenA


In [13]:
# Apply this function to clean up data, get rid of non-letters
def remove_char(x):
    """Removes non-letters and extra white space"""
    x = re.sub(r'[^a-zA-Z ]', '', x)
    x = re.sub(r'\s+', ' ', x)
    return x

df_2['Comment'] = df_2.loc[:, 'Comment'].apply(lambda x: remove_char(x))

In [14]:
# Keeping all rows that have features
df_2 = df_2.loc[(df_2['Comment'] != ' '), :]
df_2 = df_2.loc[(df_2['Comment'] != ''), :]
df_2.head()

,Title,Platform,Userscore,Comment,Username
0,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Everything in OoT is so near at perfection its...,SirCaestus
1,The Legend of Zelda: Ocarina of Time,Nintendo64,10,I wont bore you with what everyone is already ...,Kaistlin
2,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Anyone who gives the masterpiece below a or an...,Jacody
3,The Legend of Zelda: Ocarina of Time,Nintendo64,10,Im one of those people who think that this is ...,doodlerman
4,The Legend of Zelda: Ocarina of Time,Nintendo64,10,This game is the highest rated game on Metacr...,StevenA


In [15]:
df_2 = df_2.iloc[:10000, :]

In [16]:
# Make sure the review is in English
df_2['Language'] = df_2.loc[:, 'Comment'].apply(lambda x: detect(x.decode('utf8')))
df_2 = df_2.loc[df_2['Language'] == 'en']

In [48]:
# Merging Reviews with earlier DataFrame, resetting its index to join on name of games
df = df.reset_index()
df_w = pd.merge(df, df_2, left_on='Name', right_on='Title')
df_w.iloc[2000:20150]

,Name,Platform_x,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,Sega Dreamcast,Xbox,Xbox 360,Xbox One,Title,Platform_y,Userscore,Comment,Username,Language
2000,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,21.04,...,0,0,0,0,Grand Theft Auto V,Xbox360,0,I love this game its one of the best ive ever ...,ILoveNerdCubed,en
2001,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,21.04,...,0,0,0,0,Grand Theft Auto V,Xbox360,0,Again that same like in old versions GTA Huge ...,Uncle,en
2002,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,21.04,...,0,0,0,0,Grand Theft Auto V,Xbox360,0,This review contains spoilers click expand to...,GTA5SUCKS,en
2003,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,21.04,...,0,0,0,0,Grand Theft Auto V,Xbox360,7,Despite the banning of many many accounts for ...,TheScientist77,en
2004,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,21.04,...,0,0,0,0,Grand Theft Auto V,Xbox360,0,it was the worst gta ever the game play sucked...,commandershepar,en
2005,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,21.04,...,0,0,0,0,Grand Theft Auto V,Xbox360,10,One of the best games I have ever played and i...,bluemonkey93,en
2006,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,21.04,...,0,0,0,0,Grand Theft Auto V,Xbox360,10,Rockstar Release GTA V on PCBecause GTA is an ...,mandala11,en
2007,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,21.04,...,0,0,0,0,Grand Theft Auto V,Xbox360,0,A pretty sucks balls game is already overvalue...,gtavsucksballs,en
2008,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,21.04,...,0,0,0,0,Grand Theft Auto V,Xbox360,10,Now on about the th main story mission stolen ...,Scritty,en
2009,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,21.04,...,0,0,0,0,Grand Theft Auto V,Xbox360,10,Its hard to write a review for a game like thi...,VidsRule,en


In [49]:
# No NaNs in this DataFrame
df_w.isnull().values.any()

False

In [50]:
df_w['Name'].value_counts()

Grand Theft Auto V          13635
Grand Theft Auto IV          4377
Super Mario Galaxy            607
Grand Theft Auto III          428
Super Mario Galaxy 2          292
Tony Hawk's Pro Skater 3      285
Metroid Prime                 274
Tony Hawk's Pro Skater 2       98
SoulCalibur                    61
Name: Name, dtype: int64

In [51]:
# Removing duplicate or unnecessary columns, renaming others for clarity
df_w = df_w.drop(['Platform_y'], axis=1)
df_w = df_w.drop(['Title'], axis=1)
df_w = df_w.drop(['Language'], axis=1)
df_w.rename(columns={'Platform_x':'Platform'}, inplace=True)
df_w.rename(columns={'User_Score':'Average_User_Score'}, inplace=True)
df_w.rename(columns={'Userscore':'Individual_User_Score'}, inplace=True)
df_w.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,PlayStation 4,PlayStation Vita,PSP,Sega Dreamcast,Xbox,Xbox 360,Xbox One,Individual_User_Score,Comment,Username
0,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,21.04,...,0,0,0,0,0,0,0,10,Great Game The best GTA now enhanced by the Xb...,rossco98
1,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,21.04,...,0,0,0,0,0,0,0,8,The updated version of this great game is a de...,Jwilco
2,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,21.04,...,0,0,0,0,0,0,0,7,This is the first GTA I have ever played and a...,Daelus
3,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,21.04,...,0,0,0,0,0,0,0,10,My best game so far Rockstar did what they hav...,Unpleased
4,Grand Theft Auto V,PS3,2013,Action,Take-Two Interactive,7.02,9.09,0.98,3.96,21.04,...,0,0,0,0,0,0,0,9,One of the games that could live up to the hyp...,Honoral


In [52]:
# First Matrix is one review per row with each column being a token
# Function to tokenize words from Video Game Reviews
def sent_tokenizer(x):
    """Tokenizes words, removes stops words, breaks down the stems and counts the tokens"""
    x = x.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    w_tokens = tokenizer.tokenize(x)
    sr = stopwords.words('english')
    for token in w_tokens:
        if token in sr:
            w_tokens.remove(token)
    ps = PorterStemmer()
    w_tokens2 = [ps.stem(w) for w in w_tokens]
    w_count = Counter(w_tokens2)
    return w_count, w_tokens2

# Creating a dictionary of dictionaries with words and their counts
tokens = df_w['Comment'].apply(lambda x: sent_tokenizer(x)[0])
tokens

0        {u'origin': 2, u'real': 1, u'code': 2, u'versi...
1        {u'enjoy': 4, u'vehicl': 2, u'is': 2, u'it': 3...
2        {u'all': 2, u'concept': 1, u'shot': 1, u'money...
3        {u'rockstar': 1, u'feel': 1, u'frame': 1, u'am...
4        {u'behav': 1, u'random': 1, u'find': 1, u'miss...
5        {u'enjoy': 1, u'pound': 1, u'dont': 1, u'feel'...
6        {u'theyr': 2, u'love': 2, u'rockstar': 4, u'fe...
7        {u'would': 2, u'identifi': 2, u'thi': 1, u'is'...
8        {u'all': 1, u'just': 1, u'money': 5, u'dollar'...
9        {u'shop': 2, u'golast': 1, u'there': 2, u'love...
10       {u'ad': 2, u'be': 2, u'itself': 2, u'touch': 2...
11       {u'just': 3, u'money': 3, u'consum': 2, u'perf...
12       {u'shot': 1, u'help': 1, u'just': 1, u'show': ...
13       {u'arcad': 3, u'ive': 2, u'all': 1, u'love': 1...
14       {u'streak': 2, u'lifeliketh': 2, u'wasnt': 2, ...
15       {u'gta': 1, u'graphic': 1, u'ad': 2, u'get': 1...
16       {u'play': 1, u'phenomen': 1, u'especi': 1, u'm.

In [53]:
# Making a list of the dictionary
tokens = list(tokens)

In [57]:
# Creating a DataFrame of the list
df_t = pd.DataFrame(tokens)
df_t.head()

,a,aa,aaa,aaatitl,aadmittedli,aafter,aagain,aaround,aat,ab,...,zentornoentityadd,zero,zerog,zev,zillion,zit,zombi,zomgbestgameevarzomgithinkimgonnaplotz,zone,zoom
0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# Naming the index, changing missing values to 0
df_t.index.name = 'Reviews'
df_t = df_t.replace(pd.np.nan, 0)
df_t.head()

,a,aa,aaa,aaatitl,aadmittedli,aafter,aagain,aaround,aat,ab,...,zentornoentityadd,zero,zerog,zev,zillion,zit,zombi,zomgbestgameevarzomgithinkimgonnaplotz,zone,zoom
Reviews,,,,,,,,,,,,,,,,,,,,,
0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
# Putting DataFrame through a TF-IDF Transformer
transformer = TfidfTransformer()
matrix = transformer.fit_transform(df_t)
matrix = matrix.todense()
df_wt = pd.DataFrame(matrix, columns=df_t.columns)
df_wt.index.name = 'Reviews'
df_wt.head()

,a,aa,aaa,aaatitl,aadmittedli,aafter,aagain,aaround,aat,ab,...,zentornoentityadd,zero,zerog,zev,zillion,zit,zombi,zomgbestgameevarzomgithinkimgonnaplotz,zone,zoom
Reviews,,,,,,,,,,,,,,,,,,,,,
0,0.138390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.093980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.081988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.065755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.070013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Get average value for each token in each game, make it one game per row (using aggregation)
# Possibly for Merging
#df_wt = df_wt.reset_index()
#df_wt.groupby(df_wt['Comment'])

In [ ]:
# Saving DataFame to CSV if needed
#df_wt.to_csv('vg_nlp_df.csv', sep='\t')

In [ ]:
# Uploading third Dataset and inspecting its contents
df_3 = pd.DataFrame()
df_3 = pd.read_csv('/Users/dwreiter/Desktop/Work/Springboard/Video Game Capstone Project/Data Wrangling/Managerial_and_Decision_Economics_2013_Video_Games_Dataset.csv', index_col=0)
df_3.head()

In [ ]:
# No NaNs in this DataFrame
df_3.isnull().values.any()

In [ ]:
# Saving DataFame to CSV
df_3.to_csv('vg_2_df.csv', sep='\t')